#### Pandas

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import pickle
# import shap

import statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor 

import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, datasets, linear_model, metrics

import seaborn as sns

import datetime as dt
from datetime import datetime
from datetime import date
import math

# TF-IDF:
import sklearn.feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve

from matplotlib.collections import LineCollection

import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
# ************************ Trigger model data: **************************

# df_targ = spark.read.table('b2b_stg.df_churn_weekly_retro_scoring_v01') 

# df_targ_pd = df_targ.toPandas()
# df_targ_pd.to_pickle('df_targ_pd.pkl')

df_targ_pd = pd.read_pickle('df_targ_pd.pkl')

print(df_targ_pd.shape)
df_targ_pd.head(2)

# (49 478 217, 3)

(49478217, 3)


,subscriber_sk,preds_trig_model,week_of_year
0,5948609,0.000347,38
1,6166117,0.000069,38


In [3]:
# ************************ ML model data: **************************

# X_hist.to_pickle('df_ml_hist.pkl')
X_hist = pd.read_pickle('df_ml_hist.pkl')

print(X_hist.shape)
X_hist.head(2)

(62626668, 5)


,subscriber_sk,week_of_year,churn_flg,target,score_ML
0,254,22,0.0,0.0,0.000097
1,254,23,0.0,0.0,0.000117


In [4]:
df_targ_ml = df_targ_pd.merge(X_hist, on = ['subscriber_sk', 'week_of_year'], how = 'left')

print(df_targ_ml.shape)
df_targ_ml.head(2)

(49479797, 6)


,subscriber_sk,preds_trig_model,week_of_year,churn_flg,target,score_ML
0,5948609,0.000347,38,NaN,NaN,NaN
1,6166117,0.000069,38,NaN,NaN,NaN


In [5]:
49479797 - 49478217

1580

In [6]:
df_targ_ml['score_ML'].isna().sum()

10536164

In [7]:
10536164 / 49479797

0.21293870708483303

In [8]:
df_targ_ml[df_targ_ml['score_ML'].isna()]

,subscriber_sk,preds_trig_model,week_of_year,churn_flg,target,score_ML
0,5948609,0.000347,38,NaN,NaN,NaN
1,6166117,0.000069,38,NaN,NaN,NaN
2,5847144,0.002399,38,NaN,NaN,NaN
3,7192983,0.000079,38,NaN,NaN,NaN
4,7267461,0.000104,38,NaN,NaN,NaN
...,...,...,...,...,...,...
49479679,562212259,0.000140,26,NaN,NaN,NaN
49479688,562211984,0.000119,26,NaN,NaN,NaN
49479711,562659283,0.002399,26,NaN,NaN,NaN
49479783,562197638,0.000140,26,NaN,NaN,NaN


In [9]:
df_targ_ml = df_targ_ml[~df_targ_ml['score_ML'].isna()]
print(df_targ_ml.shape)

(38943633, 6)


In [10]:
49479797 - 10536164

38943633

In [11]:
df_targ_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38943633 entries, 8391936 to 49479795
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   subscriber_sk     int64  
 1   preds_trig_model  float64
 2   week_of_year      int64  
 3   churn_flg         float64
 4   target            float64
 5   score_ML          float64
dtypes: float64(4), int64(2)
memory usage: 2.0 GB


#### LogisticRegression

In [12]:
from sklearn.linear_model import LogisticRegression, LinearRegression

In [13]:
X = df_targ_ml[['preds_trig_model', 'score_ML']]
y = df_targ_ml['target']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

print(X_train.shape, X_test.shape)
print(y_train.mean(), y_test.mean())

(31154906, 2) (7788727, 2)
0.00041990176442836965 0.0004198375421297986


In [15]:
# ******* LGBMClassifier: *****

clf_lgb_2 = lgb.LGBMClassifier() 

clf_lgb_2.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 13082, number of negative: 31141824
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.064601 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 31154906, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000420 -> initscore=-7.775070
[LightGBM] [Info] Start training from score -7.775070


LGBMClassifier()

In [16]:
y_pred_test = clf_lgb_2.predict_proba(X_test)[:,1]

AUC_test = metrics.roc_auc_score(y_test, y_pred_test)
Gini_test = 2 * AUC_test - 1
print('AUC_test:', AUC_test)
print('Gini_test:', Gini_test)

# *** 26 features (not balanced): ***
# AUC_test: 0.8835047059892449
# Gini_test: 0.7670094119784898

AUC_test: 0.847433957923774
Gini_test: 0.6948679158475479


In [17]:
0.8835047059892449 - 0.847433957923774

0.03607074806547095

In [18]:
y_pred = clf_lgb_2.predict_proba(X)[:,1]

AUC = metrics.roc_auc_score(y, y_pred)
Gini = 2 * AUC - 1
print('AUC:', AUC)
print('Gini:', Gini)

AUC: 0.8535459248392485
Gini: 0.707091849678497


In [19]:
# ************** LogisticRegression: **************

log_reg = LogisticRegression().fit(X_train, y_train)

y_pred_test_log = log_reg.predict_proba(X_test)[:,1]

AUC_test = metrics.roc_auc_score(y_test, y_pred_test_log)
Gini_test = 2 * AUC_test - 1

print('AUC_test:', AUC_test)
print('Gini_test:', Gini_test)

# boosting:
# AUC_test: 0.847433957923774
# Gini_test: 0.6948679158475479

# log_reg:
# AUC_test: 0.8272859026796175
# Gini_test: 0.6545718053592351

AUC_test: 0.8272859026796175
Gini_test: 0.6545718053592351


In [20]:
y_pred_log = log_reg.predict_proba(X)[:,1]

AUC = metrics.roc_auc_score(y, y_pred_log)
Gini = 2 * AUC - 1
print('AUC:', AUC)
print('Gini:', Gini)

# boosting:
# AUC: 0.8535459248392485
# Gini: 0.707091849678497

# log_reg:
# AUC: 0.8274296568124014
# Gini: 0.6548593136248029

AUC: 0.8274296568124014
Gini: 0.6548593136248029


In [21]:
print(log_reg.coef_, log_reg.intercept_)

[[-0.00173262  0.01152435]] [-7.60430242]


In [ ]:
# ********* save model 22.10.25 (aggregation logreg): *****************

# with open('model_logreg_aggregate.pkl', 'wb') as fid:

#         pickle.dump(log_reg, fid)

# ********************** read: *******************************

with open('model_logreg_aggregate.pkl', 'rb') as fid:
    
    log_reg = pickle.load(fid)

print(log_reg.coef_, log_reg.intercept_)
# [[-0.00173262  0.01152435]] [-7.60430242]

#### Linear regression

In [18]:
# ************** LinearRegression: **************

lin_reg = LinearRegression().fit(X_train, y_train)

y_pred_test_lin = lin_reg.predict(X_test)

from sklearn.metrics import r2_score, mean_squared_error

r2 = r2_score(y_test, y_pred_test_lin)
print('r2:', r2)

mse = mean_squared_error(y_test, y_pred_test_lin)
print('mse:', mse)

# boosting:
# AUC_test: 0.847433957923774
# Gini_test: 0.6948679158475479

# log_reg:
# AUC_test: 0.8272859026796175
# Gini_test: 0.6545718053592351

r2: 0.0009184258617064289
mse: 0.00041927585079662194


In [21]:
print(lin_reg.coef_, lin_reg.intercept_)

[0.51901839 0.17344183] 0.00013529587971407092


In [24]:
import pickle

# *************** save model 23.10.25: *****************

# with open('model_linreg_aggregate.pkl', 'wb') as fid:

#         pickle.dump(lin_reg, fid)

# ********************** read: ***********************

with open('model_linreg_aggregate.pkl', 'rb') as fid:
    
    lin_reg = pickle.load(fid)

print(lin_reg.coef_, lin_reg.intercept_)

# [0.51901839 0.17344183] 0.00013529587971407092

[0.51901839 0.17344183] 0.00013529587971407092
